In [1]:
import os
os.remove('snorkel.db')
os.system('cp snorkel.db\ corpus snorkel.db');

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from snorkel import SnorkelSession
session = SnorkelSession()

import os, sys
sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/tables/')

### Extraction

In [3]:
from snorkel.models import Corpus

corpus = session.query(Corpus).filter(Corpus.name == 'Hardware').one()
print "%s contains %d Documents" % (corpus, len(corpus))

Corpus (Hardware) contains 100 Documents


In [4]:
from snorkel.models import candidate_subclass

Part = candidate_subclass('Part', ['part'])

In [5]:
from hardware_utils import OmniNgramsHardware

hardware_ngrams = OmniNgramsHardware(n_max=3, split_tokens=None)

In [6]:
from hardware_utils import load_extended_parts_dict
from snorkel.matchers import DictionaryMatch

filename = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
parts_dict = load_extended_parts_dict(filename)
print "Loaded %d part numbers." % len(parts_dict)

parts_matcher = DictionaryMatch(d=parts_dict, ignore_case=True)

Loaded 1286 part numbers.


In [7]:
from snorkel.candidates import CandidateExtractor

ce = CandidateExtractor(Part, [hardware_ngrams], [parts_matcher])

In [8]:
%time candidates = ce.extract(corpus.documents, 'Hardware Candidates', session)
print "%s contains %d Candidates" % (candidates, len(candidates))

[========================================] 100%

CPU times: user 2min 47s, sys: 3.29 s, total: 2min 50s
Wall time: 2min 52s
Candidate Set (Hardware Candidates) contains 16070 Candidates


### Saving Candidates

In [9]:
session.add(candidates)
session.commit()

In [10]:
import os
os.system('cp snorkel.db snorkel.db\ candidates');

### Reloading Candidates

In [11]:
from snorkel import SnorkelSession
session = SnorkelSession()

import os, sys
sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/tables/')

import os
os.remove('snorkel.db')
os.system('cp snorkel.db\ candidates snorkel.db');

from snorkel.models import Corpus
corpus = session.query(Corpus).filter(Corpus.name == 'Hardware').one()

In [12]:
from snorkel.models import CandidateSet

candidates = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Candidates').one()
print "%s contains %d Candidates" % (candidates, len(candidates))

Candidate Set (Hardware Candidates) contains 16070 Candidates


### Error Analysis

In [13]:
# TEMP
from hardware_utils import entity_level_total_recall

filename = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
(tp, fp, fn) = entity_level_total_recall(candidates, filename, 'stg_temp_min')

Preparing gold set...
Preparing candidates...
[========================================] 100%

Scoring on Entity-Level Total Recall
Entity-level Candidates extracted: 2087 
Entity-level Gold: 542
Intersection Candidates: 528
----------------------------------------
Overlap with Gold:  0.97



In [14]:
for c in list(fn):
    print c

('SIEMS01215-1', 'BC857A')
('PJECS00521-1', 'MMBT3904')
('SIEMS01215-1', 'BC858C')
('SIEMS01215-1', 'BC858B')
('SIEMS01215-1', 'BC859B')
('SIEMS01215-1', 'BC858A')
('SIEMS01215-1', 'BC856A')
('SIEMS01215-1', 'BC859C')
('SIEMS01215-1', 'BC860B')
('SIEMS01215-1', 'BC859A')
('SIEMS01215-1', 'BC856B')
('SIEMS01215-1', 'BC857B')
('SIEMS01215-1', 'BC860C')
('SIEMS01215-1', 'BC857C')


In [30]:
print candidates[0].part.parent.document.name

MCCCS08984-1


In [54]:
from hardware_utils import expand_part_range

for p in expand_part_range("BC 856 A"): print p

BC856
BC856-16
BC856-25
BC856-40
BC856A
BC856B
BC856C


In [23]:
"BC 856 A" in parts_dict

True

In [45]:
for c in candidates:
    if c.part.parent.document.name == "SIEMS01215-1":
        import pdb; pdb.set_trace()

KeyboardInterrupt: 

In [16]:
# for c in candidates:
#     if c.part.get_span() == 'BC858A':
#         import pdb; pdb.set_trace()

In [17]:
# def get_doc(corpus, doc_name):
#     for doc in corpus.documents:
#         if doc.name.upper() == doc_name.upper():
#             return doc

In [46]:
for c in candidates[100:200]:
    print c

Part(ImplicitSpan("BC857C", parent=129929, words=[0,0], position=[6]))
Part(Span("BC859", parent=129649, chars=[0,4], words=[0,0]))
Part(Span("BC860", parent=9906, chars=[12,16], words=[2,2]))
Part(ImplicitSpan("BC857", parent=129457, words=[0,0], position=[0]))
Part(ImplicitSpan("BC860A", parent=129932, words=[0,0], position=[4]))
Part(ImplicitSpan("BC859A", parent=129649, words=[0,0], position=[4]))
Part(ImplicitSpan("BC857", parent=9907, words=[0,0], position=[0]))
Part(ImplicitSpan("BC858B", parent=129930, words=[0,0], position=[5]))
Part(Span("BC858", parent=88596, chars=[6,10], words=[1,1]))
Part(ImplicitSpan("BC856B", parent=88596, words=[0,0], position=[5]))
Part(Span("BC856", parent=9906, chars=[0,4], words=[0,0]))
Part(Span("BC860A-Z4E", parent=129447, chars=[0,9], words=[0,0]))
Part(ImplicitSpan("BC857A", parent=129457, words=[0,0], position=[4]))
Part(Span("BC857", parent=9907, chars=[0,4], words=[0,0]))
Part(Span("BC856", parent=129928, chars=[0,4], words=[0,0]))
Part(Span

In [19]:
# doc = get_doc(corpus, 'Infineon-BC857SERIES_BC858SERIES_BC859SERIES_BC860SERIES-DS-v01_01-en')
# for phrase in doc.phrases:
#     if 'BC857CW' in phrase.words:
#         import pdb; pdb.set_trace()